In [13]:
import mbuild as mb
import os
from parmed import load_file
import re
from mbuild.lib.atoms import H
import numpy as np
import random
import pubchempy as pcp
from rdkit import Chem 
from Main_functions.crosslink_function import *

The first step is loading the functions from the crosslink_function.py

In this notebook, we explore the crosslinking process of poly(vinyl alcohol) (PVA) with various crosslinkers.
Specifically, we will examine two examples in detail: glutaraldehyde and dimethylolurea.
The number of crosslinkers that can be used can be easily extended by following the logic implemented in the code.

First, we will retrieve the SMILES string of the compound from the PubChem database.
Then, we will use mBuild to convert the SMILES into an mb.Compound() object and visualize it using a set of functions provided in crosslink_functions.py.

In [14]:
_, cid = fetch_and_plug_CR("Glutaraldehyde")
smiles, _ = get_mol_CR(cid)
crosslinker1 = Build_3D_CR(smiles) 
crosslinker1.visualize()

Base name: Glutaraldehyde, SMILES: None, CID: 3485
C(CC=O)CC=O


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now, let's do the same for Dimethylolurea

In [15]:
_, cid = fetch_and_plug_CR("Dimethylolurea")
smiles, _ = get_mol_CR(cid)
crosslinker2 = Build_3D_CR(smiles) 
crosslinker2.visualize()

Base name: Dimethylolurea, SMILES: None, CID: 8827
C(NC(=O)NCO)O


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Using part of the Mbuild functionality and the Polymer class, let's build and visualize a PVA polymer chain with 12 repeat units.
For this, first we will build the repeat unit of PVA:
1. First we will create an ethanol molecule
2. We will remove atoms and add ports in order to create the monomer that will constitute the polymer chain

In [16]:
main_name = "ethanol"
MB_molecule_to_create_monomer = Alcohol(chain_length=2)                                 
first_port_monomer = "COH"
second_port_monomer = "CH3"

main_polymer_bead, end_bead = Builder_pipeline_Polymer(main_name,
                                                       first_port = first_port_monomer,
                                                       second_port = second_port_monomer,
                                                       add_ports = True,
                                                       rigid_port = False,
                                                       fetch_compound = False,
                                                       MB_compound = MB_molecule_to_create_monomer,
                                                       factor_length = 1)


repeat_units = 12 # Repeat Units in polymer chain
polymer_compound_ = mb.recipes.Polymer(monomers=[main_polymer_bead])
polymer_compound_.build( n=repeat_units,)
main_polymer_bead.visualize()

Polymer loaded as mb compound, no query needed
COH
The port is type: COH
Added ports to the molecule
Added 2 ports to the molecule


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Alternatively, the polymer repeat unit can be prepared as a .mol2 or .pdb file and loaded into an mBuild compound, which can then be passed to the Polymer class. This approach enables more careful handling of complex chemistries. For a detailed example of this process, see the Creating_polymer.ipynb notebook in this repository.

In [17]:

repeat_units = 12 # Repeat Units in polymer chain
polymer_compound_ = mb.recipes.Polymer(monomers=[main_polymer_bead])
polymer_compound_.build( n=repeat_units,)
polymer_compound_.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The main difference between the crosslinkers Glutaraldehyde and DImethylolurea, is that 
1. Glutaraldehyde attaches to 2 oxygens in each polymer chain
    a) With a total of 4 bonds per crosslinker molecule
2. Dimethylolurea attaches to 1 oxygen in each polymer chain 
    b) With a tital of 2 bonds per crosslinker molecule

Definition of a Crosslinker (and Its Bonding to Polymer Chains)

The bonding behavior of a crosslinker is defined by the following aspects:

    Anchor Atom: The atom in the crosslinker molecule that bonds to the main polymer chain (e.g., an aldehyde carbon or nitrogen).

    Anchor Atom – Number of Bonds: The number of bonds the anchor atom forms.

    Target Atoms: The atoms directly bonded to the anchor atom that should be removed when the bond to the polymer chain is created.

    Extra Pruning (prune_extra flag): Indicates whether atoms bonded to the target atoms should also be removed.
    For example, if the target atom is an oxygen from an –OH group, the bonded hydrogen must also be removed.

    Crosslink Sites per Molecule (crosslink_sites_per_molecule):
    Indicates how many anchor sites exist per crosslinker molecule.
    Examples:

        Glutaraldehyde: 2

        Dimethylolurea: 1



In [18]:
Crosslinker_types = {

"Glutaraldehyde" : {

    "anchor_atom" : "C",
    "target_atom" : ["H", "O"],
    "anchor_atom_n_bonds" : 3,
    "extra_pruning": [],
    "exclude": ["H"],
    "prune_extra_flag": False,
    "crosslink_sites_per_molecule": 2,
    },

"Dimethylolurea" : {

    "anchor_atom" : "C",
    "target_atom" : ["O",],
    "anchor_atom_n_bonds" : 4,
    "extra_pruning": ["O",["H"]],
    "exclude": [],
    "prune_extra_flag": True,
    "crosslink_sites_per_molecule": 1,
    },

"Malic acid" : {

    "anchor_atom" : "C",
    "target_atom" : ["O",],
    "anchor_atom_n_bonds" : 3,
    "extra_pruning": ["O",["H"]],
    "exclude": [],
    "prune_extra_flag": False,
    "crosslink_sites_per_molecule": 1,
    },


"Oxalic acid":

{

    "anchor_atom" : "O",
    "target_atom" : ["H",],
    "anchor_atom_n_bonds" : 2,
    "extra_pruning": [],
    "exclude": [],
    "prune_extra_flag": False,
    "crosslink_sites_per_molecule": 1,
    },



}



In [19]:
crosslinker_types = [ 
    "Glutaraldehyde",
    "Dimethylolurea",
]

In [20]:

crosslinker_molecule_targets = Crosslinker_types[crosslinker_types[0]]

Compound, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
    
    main_name = "ethanol",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    MB_molecule_to_create_monomer = Alcohol(chain_length=2), # Mbuild compound, ethanol molecule
    fetch_compound_from_database = False,
    first_port_monomer = "COH",
    second_port_monomer = "CH3",
    
    repeat_units = 12,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 1.3,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                            #Crosslinker details
    crosslinker_name = crosslinker_types[0],                      # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports


                                                            #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 60,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "O",
                            "target_atom" : "H",
                            "bonded_atoms": ["C", "H"], 
                            "anchor_atom_n_bonds" : 2,
                            },
                            ],
    
    crosslinker_backbone_target_list = [
                            {
                            "anchor_atom" : crosslinker_molecule_targets["anchor_atom"],
                            "target_atom" : crosslinker_molecule_targets["target_atom"],
                            "anchor_atom_n_bonds" :crosslinker_molecule_targets["anchor_atom_n_bonds"],
                            "extra_pruning":crosslinker_molecule_targets["extra_pruning"],
                            "exclude":crosslinker_molecule_targets["exclude"],
                            },
                            ],

                
    
    prune_extra_flag =crosslinker_molecule_targets["prune_extra_flag"],                                  # No pruning extra atoms 
    crosslink_sites_per_molecule = crosslinker_molecule_targets["crosslink_sites_per_molecule"],                      # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
    
)

print(Real_Degree_Crosslinking)
print(Polymer_Residue_Name)
print(Crosslinker_Resiude_Name)
print(Number_Crosslinkers_Used)

Compound.visualize(show_ports = False)

Number of ports per chain 6
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 22, 2: 44, 3: 66}
To attain the degree of crosslinking 60% 
 6 crosslinkers are needed 
 3 between each polymer chain
The actual attainable value of crosslinking is 66%
Which is the highest crosslinking percentage attainable
Polymer loaded as mb compound, no query needed
COH
The port is type: COH
Added ports to the molecule
Added 2 ports to the molecule
Done with loading compounds
Base name: Glutaraldehyde, SMILES: None, CID: 3485
C(CC=O)CC=O
{'POL-0': [(0, array([-1.72640950e-17,  1.45000000e-01,  3.44485747e-18])), (1, array([-0.2616,  0.145 ,  0.14  ])), (2, array([-0.5232,  0.145 ,  0.28  ])), (3, array([-0.7848,  0.145 ,  0.42  ])), (4, array([-1.0464,  0.145 ,  0.56  ])), (5, array([-1.308,  0.145,  0.7  ])), (6, array([-1.5696,  0.145 ,  0.84  ])), (7, array([-1.8312,  0.145 ,  0.98  ])), (8, array([-2.092

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [21]:

crosslinker_molecule_targets = Crosslinker_types[crosslinker_types[1]]

Compound, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
    
    main_name = "ethanol",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    MB_molecule_to_create_monomer = Alcohol(chain_length=2), # Mbuild compound, ethanol molecule
    fetch_compound_from_database = False,
    first_port_monomer = "COH",
    second_port_monomer = "CH3",
    
    repeat_units = 12,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 1.3,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                            #Crosslinker details
    crosslinker_name = crosslinker_types[1],                      # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports


                                                            #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 60,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "O",
                            "target_atom" : "H",
                            "bonded_atoms": ["C", "H"], 
                            "anchor_atom_n_bonds" : 2,
                            },
                            ],
    
    crosslinker_backbone_target_list = [
                            {
                            "anchor_atom" : crosslinker_molecule_targets["anchor_atom"],
                            "target_atom" : crosslinker_molecule_targets["target_atom"],
                            "anchor_atom_n_bonds" :crosslinker_molecule_targets["anchor_atom_n_bonds"],
                            "extra_pruning":crosslinker_molecule_targets["extra_pruning"],
                            "exclude":crosslinker_molecule_targets["exclude"],
                            },
                            ],

                
    
    prune_extra_flag =crosslinker_molecule_targets["prune_extra_flag"],                                  # No pruning extra atoms 
    crosslink_sites_per_molecule = crosslinker_molecule_targets["crosslink_sites_per_molecule"],                      # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
    
)

print(Real_Degree_Crosslinking)
print(Polymer_Residue_Name)
print(Crosslinker_Resiude_Name)
print(Number_Crosslinkers_Used)

Compound.visualize(show_ports = False)

Number of ports per chain 5
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 11, 2: 22, 3: 33, 4: 44, 5: 55, 6: 66}
To attain the degree of crosslinking 60% 
 10 crosslinkers are needed 
 5 between each polymer chain
The actual attainable value of crosslinking is 55%
Polymer loaded as mb compound, no query needed
COH
The port is type: COH
Added ports to the molecule
Added 2 ports to the molecule
Done with loading compounds
Base name: Dimethylolurea, SMILES: None, CID: 8827
C(NC(=O)NCO)O
{'POL-0': [(0, array([-1.72640950e-17,  1.45000000e-01,  3.44485747e-18])), (1, array([-0.2616,  0.145 ,  0.14  ])), (2, array([-0.5232,  0.145 ,  0.28  ])), (3, array([-0.7848,  0.145 ,  0.42  ])), (4, array([-1.0464,  0.145 ,  0.56  ])), (5, array([-1.308,  0.145,  0.7  ])), (6, array([-1.5696,  0.145 ,  0.84  ])), (7, array([-1.8312,  0.145 ,  0.98  ])), (8, array([-2.0928,  0.145 ,  1.12  ])), (9, arra

3Dmol.js failed to load for some reason. Please check your browser console for error messages.